In [19]:
!pip install -U dash --quiet
!pip install plotly --quiet

622.92s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
629.92s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [16]:
import pandas as pd
df = pd.read_csv('election_results.csv')
df

,State,State Code,District,Party,Candidate,Incumbent,Vote,Pct%
0,Alabama,AL,1st,Republican,Carl,Incumbent,139854,84.2%
1,Alabama,AL,1st,Libertarian Party,Remrey,NaN,26197,15.8%
2,Alabama,AL,2nd,Republican,Moore,Incumbent,137193,69.2%
3,Alabama,AL,2nd,Democratic,Harvey-Hall,NaN,57800,29.1%
4,Alabama,AL,2nd,Libertarian Party,Realz,NaN,3380,1.7%
...,...,...,...,...,...,...,...,...
1052,Wisconsin,WI,8th,Libertarian Party,VandenPlas,NaN,32043,10.5%
1053,Wyoming,WY,1st,Republican,Harriet Hageman,NaN,132215,69.8%
1054,Wyoming,WY,1st,Democratic,Lynnette GreyBull,NaN,47241,24.9%
1055,Wyoming,WY,1st,Libertarian Party,Richard Brubaker,NaN,5420,2.9%


In [17]:
# data pre process
import numpy as np

df = df[df['Party'] != 'no party']
df = df[df['Party'] != 'other']
df['Party'] = df['Party'].apply(lambda x: "Libertarian" if x == "Libertarian Party" else x)

state_list = df['State'].unique()
state_list = np.insert(state_list,0, 'All States')

winner_df = df[df.groupby(['State', 'District'])['Vote'].transform(max)==df['Vote']].copy()
winner_df["District"] = winner_df["District"].apply(lambda x: x[:-2].zfill(2) if isinstance(x, str) and len(x) >= 2 else x)
winner_df['District'] = winner_df['State'] + ' ' + winner_df['District']
winner_df['Pct'] = winner_df['Pct%'].str.replace('%','').fillna(0).astype(float)
winner_df['Pct'] = winner_df['Pct'].where(winner_df['Party'] != 'Democratic', -winner_df['Pct'])

C:\Users\11066\AppData\Local\Temp\ipykernel_63808\3815706768.py:11: FutureWarning:

The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.



In [18]:
# overall figure
import plotly.express as px


color_map = {'Republican' : '#ED1C2E', 'Democratic' : '#005BAC', 'Libertarian': '#FED105'}
party_seq = {'Party': ['Republican', 'Democratic', 'Libertarian'] }

def update_total_fig(value):
  cap = value
  if value == 'All States':
    grouped_total_df = df.groupby('Party')['Vote'].sum().reset_index()
  else:
    grouped_total_df = df[df['State'] == value]
    grouped_total_df = grouped_total_df.groupby('Party')['Vote'].sum().reset_index()
  total_fig = px.bar(
          grouped_total_df,
          x='Vote',
          y='Party',
          color = 'Party',
          color_discrete_map = color_map,
          orientation='h',
          category_orders = party_seq,
          title=f"Total votes recieved in {cap}")
  total_fig.update_traces(hovertemplate='<b>Votes</b>: %{x:,.0f}')
  total_fig.update_layout(xaxis=dict(tickformat=",d"), xaxis_title="Votes")
  return total_fig


#total_fig = update_total_fig('All States')
#total_fig.show()


In [19]:
# vote figure
def update_vote_fig(value):
  if value == 'All States':
    x='State'
    cap = "states"
    grouped_vote_df = df.groupby(['State','Party'])['Vote'].sum().reset_index()
  else:
    x = 'District'
    cap = "congressional districts"
    grouped_vote_df = df[df['State'] == value]
    grouped_vote_df = grouped_vote_df.groupby(['District','Party'])['Vote'].sum().reset_index()
  vote_fig = px.bar(
      grouped_vote_df,
      x=x,
      y='Vote',
      color = 'Party',
      color_discrete_map = color_map,
      category_orders = party_seq,
      title=f"Votes recieved across {cap}"
      )
  vote_fig.update_traces(hovertemplate='<b>Votes</b>: %{y:,.0f}')
  vote_fig.update_layout(xaxis=dict(title=None, dtick=1, tickangle=-50,tickvals=state_list[1:]))
  vote_fig.update_layout(yaxis=dict(tickformat=",d"), yaxis_title="Votes")
  return vote_fig

#vote_fig = update_vote_fig('All States')
#vote_fig.show()

In [20]:
# map figure
import plotly.express as px
import json

with open('json/cd118.json') as f:
    cds = json.load(f)
with open('json/statecode.json') as f:
    statecode = json.load(f)
with open('json/cdname.json') as f:
    cdname = json.load(f)

cdname = {v: k for k, v in cdname.items()}
cdname = {k.replace("at Large", "01"): v for k, v in cdname.items()}
cdname["Hawaii 01"]= "1501"
cdname["Hawaii 02"]= "1502"
winner_df['District Code'] = winner_df['District'].map(cdname)


def update_map_fig(value):
  if value=='All States':
    map_df = winner_df
    bound = None
  else:
    bound = 'locations'
    map_df = winner_df[winner_df['State'] == value]
  map_fig = px.choropleth(
            map_df,
            geojson=cds,
            scope="usa",
            locations='District Code',
            color = 'Pct',
            color_continuous_scale="RdBu_r",
            range_color=(-100, 100),
            custom_data=['Candidate', 'Party', 'Vote','District']
            )

  map_fig.update_traces(hovertemplate='<b>%{customdata[3]}</b><br><br><b>Winner</b>: <i>%{customdata[0]}</i><br><b>Winning Party</b>: <i>%{customdata[1]}</i><br><b>Votes</b>: <i>%{customdata[2]:,.0f}</i>')

  map_fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0},
    coloraxis_colorbar=dict(title=dict(text=None), x=0.95,
    tickvals=[-100, 0, 100],
    ticktext=["favor democratic", "neutral", "favor republican"],
    outlinecolor='black', outlinewidth=1,
    lenmode='fraction', len=0.3, thickness=15,
    tickfont=dict(size=14)))
  map_fig.update_geos(fitbounds= bound, visible=False)


  return map_fig

#map_fig = update_map_fig('All States')
#map_fig.show()


In [21]:
# map2 figure   extra feature
def update_map2_fig(value):
  if value=='All States':
    map2_df = winner_df
    bound = None
  else:
    bound = 'locations'
    map2_df = winner_df[winner_df['State'] == value]
  map2_fig = px.choropleth(
            map2_df,
            geojson=cds,
            scope="usa",
            locations='District Code',
            color = 'Party',
            color_discrete_map = color_map,
            custom_data=['Candidate', 'Party', 'Vote', 'District', 'Pct%']
            )

  map2_fig.update_traces(hovertemplate='%{customdata[3]}<br><b>Winning Party: %{customdata[1]}<br>Vote%: %{customdata[4]}')

  map2_fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})

  return map2_fig

#map2_fig = update_map2_fig('All States')
#map2_fig.show()

In [22]:
def update_h3(value):
  if value=="All States":
    h3_df = winner_df
  else:
    h3_df = winner_df[winner_df['State'] == value]
  total = len(h3_df)
  rp = h3_df[h3_df['Party'] == 'Republican'].shape[0]
  dm = h3_df[h3_df['Party'] == 'Democratic'].shape[0]
  h3_content = f"Total seats: {total} | Republican won: {rp} seats | Democratic won: {dm} seats"
  return h3_content

In [23]:
from dash import html, dcc, Dash
from dash.dependencies import Input, Output
import pandas as pd

app = Dash(__name__)

vote_fig = update_vote_fig('All States')
total_fig = update_total_fig('All States')
map_fig = update_map_fig('All States')
map2_fig = update_map2_fig('All States')

styles={
    'heading':{'display':'flex','justify-content':'center'},
    'map_container':{'width': '50%', 'height': '600px', 'float': 'left', 'display': 'inline'},
    'bar_container':{'width': '50%', 'float': 'right', 'display': 'inline-block'},
}

app.layout = html.Div([
             html.Div(html.H1("US House Election 2022", id='heading', style=styles['heading'])),
             html.Hr(),
             html.Div([
                 html.Div([
                     html.H3("Total seats: 435 | Republican won: 222 seats | Democratic won: 213 seats",id="seats_won"),
                     dcc.Dropdown(state_list, id='state_dropdown', placeholder="All States", value=state_list[0]),
                     dcc.Graph(id='map',figure=map_fig),
                     dcc.Graph(id='map2',figure=map2_fig)
                 ],style=styles['map_container']),

                 html.Div([
                     html.Div([
                         html.H3(id="state_result"),
                         dcc.Graph(id='party_bar', figure=total_fig)
                     ]),
                     html.Div(
                         dcc.Graph(id='party_region_bar', figure=vote_fig)
                     )
                 ],style=styles['bar_container'])
                 ])
            ])


@app.callback(
    Output('seats_won', 'children'),
    Output('map', 'figure'),
    Output('map2','figure'),
    Output('party_bar', 'figure'),
    Output('party_region_bar','figure'),
    Input('state_dropdown','value'))

def update_distrcit_list(value):
  return update_h3(value) ,update_map_fig(value),update_map2_fig(value),update_total_fig(value),update_vote_fig(value)

app.run_server(port=8091)
"""  Please open http://localhost:8091  """

'  Please open http://localhost:8091  '